<a href="https://colab.research.google.com/github/connorpheraty/DS-Project-2/blob/master/DS3_Project2_TWP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('seaborn-whitegrid')
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

In [0]:
cd DS3-Sprint Challenge

[Errno 2] No such file or directory: 'DS3-Sprint Challenge'
/Users/connorheraty/Desktop/DS3-Sprint Challenge


In [0]:
train_feature_url = 'train_features.csv'

In [0]:
train = pd.read_csv('train_features.csv', index_col='id')
test = pd.read_csv('test_features.csv', index_col='id')
target = pd.read_csv('train_labels.csv', index_col='id')

In [0]:
train.dtypes

amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_type              object
water_qual

In [0]:
train.nunique()

amount_tsh                  98
date_recorded              356
funder                    1897
gps_height                2428
installer                 2145
longitude                57516
latitude                 57517
wpt_name                 37400
num_private                 65
basin                        9
subvillage               19287
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2696
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity

In [0]:
train.isnull().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

In [0]:
# Drop non-relevant columns

train = train.drop(['wpt_name', 'num_private', 'region', 'district_code', 'ward', 'public_meeting',
                    'recorded_by', 'scheme_name', 'funder', 'subvillage', 'extraction_type_group',
                    'extraction_type_class', 'waterpoint_type_group', 'payment', 'management_group',
                   'quality_group', 'installer'], axis=1)

test = test.drop(['wpt_name', 'num_private', 'region', 'district_code', 'ward', 'public_meeting',
                  'recorded_by', 'scheme_name', 'funder', 'subvillage', 'extraction_type_group',
                  'extraction_type_class', 'waterpoint_type_group', 'payment', 'management_group',
                 'quality_group', 'installer'], axis=1)

In [0]:
# region_code --> categorical

train['region_code'] = train['region_code'].astype(str)
test['region_code'] = test['region_code'].astype(str)

In [0]:
# date_recorded --> split by year, month, and day
train[['year', 'month', 'day']] = train['date_recorded'].str.split(pat='-', n=-1, expand=True)
test[['year', 'month', 'day']] = test['date_recorded'].str.split(pat='-', n=-1, expand=True)

In [0]:
# drop date_recorded
train = train.drop('date_recorded', axis=1)
test = test.drop('date_recorded', axis=1)

In [0]:
# Transform year, month, and day into integers
train[['year', 'month', 'day']] = train[['year', 'month', 'day']].astype(int)
test[['year', 'month', 'day']] = test[['year', 'month', 'day']].astype(int)

In [0]:
# Most construction years are zero
# We will replace 0 values with inspection year to mitigate loss of data
train['construction_year'].describe()

count    59400.000000
mean      1300.652475
std        951.620547
min          0.000000
25%          0.000000
50%       1986.000000
75%       2004.000000
max       2013.000000
Name: construction_year, dtype: float64

In [0]:
train['construction_year'].loc[train['construction_year'] == 0] = train['year']
test['construction_year'].loc[test['construction_year'] == 0] = test['year']


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# Null values removed!
train['construction_year'].isnull().sum()

0

In [0]:
# Split Numeric and Categorical Columns

def split(df):
    numerics = ['int64', 'float64']
    df_num = df.select_dtypes(include=numerics)
    df_cat = df.drop(df_num, axis='columns')
    print(df.shape, df_num.shape, df_cat.shape)
    return df_num, df_cat
        

In [0]:
train_num, train_cat = split(train)
test_num, test_cat = split(test)

(59400, 24) (59400, 9) (59400, 15)
(14358, 24) (14358, 9) (14358, 15)


In [0]:
train_num.describe()

,amount_tsh,gps_height,longitude,latitude,population,construction_year,year,month,day
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,179.909983,2001.919495,2011.921667,4.375640,15.621498
std,2997.574558,693.116350,6.567432,2.946019e+00,471.482176,12.254881,0.958758,3.029247,8.687553
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1960.000000,2002.000000,1.000000,1.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,1996.000000,2011.000000,2.000000,8.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,25.000000,2008.000000,2012.000000,3.000000,16.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,215.000000,2011.000000,2013.000000,7.000000,23.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,30500.000000,2013.000000,2013.000000,12.000000,31.000000


In [0]:
train_num['age_at_inspection'] = train_num['year'] - train_num['construction_year']
test_num['age_at_inspection'] = test_num['year'] - test_num['construction_year']


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
train_num.describe().T

,count,mean,std,min,25%,50%,75%,max
amount_tsh,59400.0,317.650385,2997.574558,0.00000,0.000000,0.000000,20.000000,3.500000e+05
gps_height,59400.0,668.297239,693.116350,-90.00000,0.000000,369.000000,1319.250000,2.770000e+03
longitude,59400.0,34.077427,6.567432,0.00000,33.090347,34.908743,37.178387,4.034519e+01
latitude,59400.0,-5.706033,2.946019,-11.64944,-8.540621,-5.021597,-3.326156,-2.000000e-08
population,59400.0,179.909983,471.482176,0.00000,0.000000,25.000000,215.000000,3.050000e+04
construction_year,59400.0,2001.919495,12.254881,1960.00000,1996.000000,2008.000000,2011.000000,2.013000e+03
year,59400.0,2011.921667,0.958758,2002.00000,2011.000000,2012.000000,2013.000000,2.013000e+03
month,59400.0,4.375640,3.029247,1.00000,2.000000,3.000000,7.000000,1.200000e+01
day,59400.0,15.621498,8.687553,1.00000,8.000000,16.000000,23.000000,3.100000e+01
age_at_inspection,59400.0,10.002172,12.458074,-7.00000,0.000000,4.000000,16.000000,5.300000e+01


In [0]:
mean_lat_train = train_num['latitude'].mean()
mean_long_train = train_num['longitude'].mean()
mean_lat_test = test_num['latitude'].mean()
mean_long_test = test_num['longitude'].mean()

train_num['distance_x_height'] = np.sqrt((train_num['gps_height']**2 + train_num['longitude'] - mean_long_train)**2 + (train_num['latitude'] - mean_lat_train)**2)
test_num['distance_x_height'] = np.sqrt((test_num['gps_height']**2 + test_num['longitude'] - mean_long_test)**2 + (test_num['latitude'] - mean_lat_test)**2)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [0]:
train_num = train_num.drop(['year', 'month', 'day'], axis=1)
test_num = test_num.drop(['year', 'month', 'day'], axis=1)

In [0]:
train_num.describe().T

,count,mean,std,min,25%,50%,75%,max
amount_tsh,59400.0,317.650385,2.997575e+03,0.000000,0.000000,0.000000,2.000000e+01,3.500000e+05
gps_height,59400.0,668.297239,6.931164e+02,-90.000000,0.000000,369.000000,1.319250e+03,2.770000e+03
longitude,59400.0,34.077427,6.567432e+00,0.000000,33.090347,34.908743,3.717839e+01,4.034519e+01
latitude,59400.0,-5.706033,2.946019e+00,-11.649440,-8.540621,-5.021597,-3.326156e+00,-2.000000e-08
population,59400.0,179.909983,4.714822e+02,0.000000,0.000000,25.000000,2.150000e+02,3.050000e+04
construction_year,59400.0,2001.919495,1.225488e+01,1960.000000,1996.000000,2008.000000,2.011000e+03,2.013000e+03
age_at_inspection,59400.0,10.002172,1.245807e+01,-7.000000,0.000000,4.000000,1.600000e+01,5.300000e+01
distance_x_height,59400.0,927026.668034,1.197130e+06,0.320553,3.878755,136165.542649,1.740423e+06,7.672902e+06


In [0]:
train_num = train_num.drop('construction_year', axis=1)
test_num = test_num.drop('construction_year', axis=1)

In [0]:
train_cat.nunique()

basin                  9
region_code           27
lga                  125
scheme_management     12
permit                 2
extraction_type       18
management            12
payment_type           7
water_quality          8
quantity               5
quantity_group         5
source                10
source_type            7
source_class           3
waterpoint_type        7
dtype: int64

In [0]:
train_cat.head(15)

,basin,region_code,lga,scheme_management,permit,extraction_type,management,payment_type,water_quality,quantity,quantity_group,source,source_type,source_class,waterpoint_type
id,,,,,,,,,,,,,,,
69572,Lake Nyasa,11,Ludewa,VWC,False,gravity,vwc,annually,soft,enough,enough,spring,spring,groundwater,communal standpipe
8776,Lake Victoria,20,Serengeti,Other,True,gravity,wug,never pay,soft,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe
34310,Pangani,21,Simanjiro,VWC,True,gravity,vwc,per bucket,soft,enough,enough,dam,dam,surface,communal standpipe multiple
67743,Ruvuma / Southern Coast,90,Nanyumbu,VWC,True,submersible,vwc,never pay,soft,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple
19728,Lake Victoria,18,Karagwe,NaN,True,gravity,other,never pay,soft,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe
9944,Pangani,4,Mkinga,VWC,True,submersible,vwc,per bucket,salty,enough,enough,other,other,unknown,communal standpipe multiple
19816,Internal,17,Shinyanga Rural,VWC,True,swn 80,vwc,never pay,soft,enough,enough,machine dbh,borehole,groundwater,hand pump
54551,Lake Tanganyika,17,Kahama,NaN,True,nira/tanira,wug,unknown,milky,enough,enough,shallow well,shallow well,groundwater,hand pump
53934,Lake Tanganyika,14,Tabora Urban,VWC,True,india mark ii,vwc,never pay,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump


## Reducing High Cardinality Columns

### Funder, Installer, 

In [0]:
train_cat['installer'].value_counts().head(11)

KeyError: 'installer'

In [0]:
train_cat['installer'].isnull().sum()

3655

In [0]:
train_cat['installer'] = train_cat['installer'].str.replace('0', 'Other')
test_cat['installer'] = test_cat['installer'].str.replace('0', 'Other')

In [0]:
train_cat['installer'] = train_cat['installer'].astype(str)
test_cat['installer'] = test_cat['installer'].astype(str)

In [0]:
train_cat['installer'].nunique()

2146

In [0]:
train_cat['installer'].value_counts().head(11)

DWE                   17402
nan                    3655
Government             1825
RWE                    1206
Commu                  1060
DANIDA                 1050
KKKT                    898
Hesawa                  840
Other                   777
TCRS                    707
Central government      622
Name: installer, dtype: int64

In [0]:
def card_reducer(df):
    for _ in df:
        if _ == 'DWE':
            return 'DWE'
        elif _ == 'Government':
            return 'Government'
        elif _ == 'RWE':
            return 'RWE'
        elif _ == 'Commu':
            return 'Commu'
        elif _ == 'DANIDA':
            return 'DANIDA'
        elif _ == 'KKKT':
            return 'KKKT'
        elif _ == 'Hesawa':
            return 'Hesawa'
        elif _ == 'TCRS':
            return 'TCRS'
        elif _ == 'Central Government':
            return 'Central Government'
        elif _ == 'CES':
            return 'CES'
        else:
            return 'Other'

In [0]:
train_cat['installer'] = train_cat['installer'].map(card_reducer)
test_cat['installer'] = test_cat['installer'].map(card_reducer)

In [0]:
train_cat['installer'].tail(10)

id
13677    Other
44885    Other
40607    Other
48348    Other
11164    Other
60739    Other
27263    Other
37057    Other
31282    Other
26348    Other
Name: installer, dtype: object

## End - Exploratory Area

In [0]:
train_cat.shape, test_cat.shape

((59400, 15), (14358, 15))

In [0]:
train_num.shape, test_num.shape

((59400, 7), (14358, 7))

In [0]:
def dummy_df(category_df):
    df_dummy = pd.DataFrame()
    for col in category_df.columns:
        df_dummy = pd.concat([df_dummy, pd.get_dummies(category_df[col], drop_first=True, prefix = 'Is')], axis='columns')
    return df_dummy

In [0]:
df_dumb_train = dummy_df(train_cat)
df_dumb_test = dummy_df(test_cat)

In [0]:
df_dumb_train.shape, df_dumb_test.shape

((59400, 242), (14358, 238))

In [0]:
train_cols = list(df_dumb_train.columns)
test_cols = list(df_dumb_test.columns)

In [0]:
train, test = df_dumb_train.align(df_dumb_test, join='left', axis=1)

In [0]:
train.shape

(59400, 332)

In [0]:
# Eliminate nan values
test = test.replace(np.nan, 0)

In [0]:
X_train = pd.concat([train_num, train], axis=1)
X_test = pd.concat([test_num, test], axis=1)

In [0]:
X_train.head()

,amount_tsh,gps_height,longitude,latitude,population,age_at_inspection,distance_x_height,Is_10,Is_11,Is_12,Is_13,Is_14,Is_15,Is_16,Is_17,Is_18,Is_19,Is_2,Is_20,Is_21,Is_24,Is_3,Is_4,Is_40,Is_5,Is_6,Is_60,Is_7,Is_8,Is_80,Is_9,Is_90,Is_99,Is_Arusha Urban,Is_Babati,Is_Bagamoyo,Is_Bahi,Is_Bariadi,Is_Biharamulo,Is_Bukoba Rural,Is_Bukoba Urban,Is_Bukombe,Is_Bunda,Is_Chamwino,Is_Chato,Is_Chunya,Is_Dodoma Urban,Is_Geita,Is_Hai,Is_Hanang,Is_Handeni,Is_Igunga,Is_Ilala,Is_Ileje,Is_Ilemela,Is_Iramba,Is_Iringa Rural,Is_Kahama,Is_Karagwe,Is_Karatu,Is_Kasulu,Is_Kibaha,Is_Kibondo,Is_Kigoma Rural,Is_Kigoma Urban,Is_Kilindi,Is_Kilolo,Is_Kilombero,Is_Kilosa,Is_Kilwa,Is_Kinondoni,Is_Kisarawe,Is_Kishapu,Is_Kiteto,Is_Kondoa,Is_Kongwa,Is_Korogwe,Is_Kwimba,Is_Kyela,Is_Lake Nyasa,Is_Lake Rukwa,Is_Lake Tanganyika,Is_Lake Victoria,Is_Lindi Rural,Is_Lindi Urban,Is_Liwale,Is_Longido,Is_Ludewa,Is_Lushoto,Is_Mafia,Is_Magu,Is_Makete,Is_Manyoni,Is_Masasi,Is_Maswa,Is_Mbarali,Is_Mbeya Rural,Is_Mbinga,Is_Mbozi,Is_Mbulu,Is_Meatu,Is_Meru,Is_Misenyi,Is_Missungwi,Is_Mkinga,Is_Mkuranga,Is_Monduli,Is_Morogoro Rural,Is_Morogoro Urban,Is_Moshi Rural,Is_Moshi Urban,Is_Mpanda,Is_Mpwapwa,Is_Mtwara Rural,Is_Mtwara Urban,Is_Mufindi,Is_Muheza,Is_Muleba,Is_Musoma Rural,Is_Mvomero,Is_Mwanga,Is_Nachingwea,Is_Namtumbo,Is_Nanyumbu,Is_Newala,Is_Ngara,Is_Ngorongoro,Is_Njombe,Is_Nkasi,Is_None,Is_Nyamagana,Is_Nzega,Is_Other,Is_Pangani,Is_Pangani,Is_Pangani,Is_Pangani,Is_Parastatal,Is_Private operator,Is_Rombo,Is_Rorya,Is_Ruangwa,Is_Rufiji,Is_Rufiji,Is_Rufiji,Is_Rufiji,Is_Rungwe,Is_Ruvuma / Southern Coast,Is_SWC,Is_Same,Is_Sengerema,Is_Serengeti,Is_Shinyanga Rural,Is_Shinyanga Urban,Is_Siha,Is_Sikonge,Is_Simanjiro,Is_Singida Rural,Is_Singida Urban,Is_Songea Rural,Is_Songea Urban,Is_Sumbawanga Rural,Is_Sumbawanga Urban,Is_Tabora Urban,Is_Tandahimba,Is_Tanga,Is_Tarime,Is_Temeke,Is_True,Is_Trust,Is_Tunduru,Is_Ukerewe,Is_Ulanga,Is_Urambo,Is_Uyui,Is_VWC,Is_WUA,Is_WUG,Is_Wami / Ruvu,Is_Water Board,Is_Water authority,Is_cemo,Is_climax,Is_communal standpipe,Is_communal standpipe multiple,Is_dam,Is_dam,Is_dam,Is_dam,Is_enough,Is_enough,Is_enough,Is_enough,Is_fluoride,Is_fluoride abandoned,Is_gravity,Is_hand dtw,Is_hand pump,Is_improved spring,Is_india mark ii,Is_india mark iii,Is_insufficient,Is_insufficient,Is_insufficient,Is_insufficient,Is_ksb,Is_lake,Is_machine dbh,Is_milky,Is_mono,Is_monthly,Is_never pay,Is_nira/tanira,Is_on failure,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other - mkulima/shinyanga,Is_other - play pump,Is_other - rope pump,Is_other - school,Is_other - swn 81,Is_parastatal,Is_per bucket,Is_private operator,Is_rainwater harvesting,Is_rainwater harvesting,Is_rainwater harvesting,Is_rainwater harvesting,Is_river,Is_river/lake,Is_salty,Is_salty abandoned,Is_seasonal,Is_seasonal,Is_seasonal,Is_seasonal,Is_shallow well,Is_shallow well,Is_shallow well,Is_shallow well,Is_soft,Is_spring,Is_spring,Is_spring,Is_spring,Is_submersible,Is_surface,Is_swn 80,Is_trust,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_vwc,Is_walimi,Is_water authority,Is_water board,Is_windmill,Is_wua,Is_wug
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [0]:
np.any(np.isnan(X_train)), np.any(np.isnan(X_test))

(False, False)

In [0]:
X_train.shape, X_test.shape

((59400, 339), (14358, 339))

## Modeling

In [0]:
target.head()

,status_group
id,
69572,0
8776,0
34310,0
67743,1
19728,0


In [0]:
def label_mapper(y):
    
    if y == 0:
        return 'functional'
    if y == 1:
        return 'non functional'
    if y == 2:
        return 'functional needs repair'

In [0]:
y_train = target['status_group']

In [0]:
y_train = y_train.map(label_mapper)

In [0]:
y_train.head()

id
69572        functional
8776         functional
34310        functional
67743    non functional
19728        functional
Name: status_group, dtype: object

In [0]:
y_train.shape

(59400,)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape

((47520, 339), (11880, 339), (14358, 339), (47520,), (11880,))

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler


model = RandomForestClassifier(n_estimators=100, 
                                max_depth=30,
                                min_samples_split = 12,
                                min_samples_leaf = 1,
                                criterion = 'gini', 
                                max_features = 6, 
                                oob_score = True, 
                                random_state=42)

In [0]:
scaler = StandardScaler()
X_train_rf = scaler.fit_transform(X_train)
X_test_rf = scaler.transform(X_test)

model.fit(X_train_rf, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=34, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=17,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=True, random_state=237, verbose=0, warm_start=False)

In [0]:
y_pred_train = model.predict(X_train_rf)

y_pred = model.predict(X_test_rf)

In [0]:
from sklearn.model_selection import cross_validate

from sklearn.metrics import accuracy_score


scores = cross_validate(model,
                        X_train_rf,y_train, 
                        scoring = 'accuracy', cv=5) 

In [0]:
pd.DataFrame(scores)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


,fit_time,score_time,test_score,train_score
0,5.668819,0.218330,0.794972,0.841821
1,5.481951,0.221220,0.793898,0.840379
2,5.502585,0.219277,0.797748,0.842649
3,5.727146,0.271598,0.795854,0.841939
4,5.669712,0.220644,0.785962,0.841650


## XGBoost

In [0]:
from xgboost import XGBClassifier
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [0]:
scaler = StandardScaler()
X_train_xg = scaler.fit_transform(X_train)
X_test_xg = scaler.transform(X_test)
X_val_xg = scaler.transform(X_val)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [0]:
grboost = XGBClassifier(learning_rate=0.07, booster = 'dart', n_estimators=200, max_depth = 8)

grboost.fit(X_train_rf, y_train)

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.07,
       max_delta_step=0, max_depth=8, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [0]:
y_train_pred_boost= grboost.predict(X_train_xg)

In [0]:
y_val_pred_boost = grboost.predict(X_val_xg, ntree_limit=50)

In [0]:
accuracy_score(y_train, y_train_pred_boost)

0.8308501683501683

In [0]:
accuracy_score(y_val, y_val_pred_boost)

0.7793771043771044

## Creating a Submission

In [0]:
def create_submission(y_test_pred):
    sample_submission = pd.read_csv('sample_submission.csv')
    submission = sample_submission.copy()
    submission['status_group'] = y_test_pred
    
    now = pd.to_datetime('now')
    filename = 'CH_' + str(now).replace(' ','_')[0:-7] 
    
    submission.to_csv(f'Submissions/{filename}.csv', index=False)
    print(f'Submissions/{filename}.csv')

In [0]:
create_submission(y_pred)

Submissions/CH_2019-05-23_22:59:06.csv


array(['functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [0]:
y_pred_df = pd.DataFrame(y_pred)

In [0]:
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred

In [0]:
submission.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [0]:
submission.status_group.value_counts(normalize=True)

functional                 0.662488
non functional             0.314320
functional needs repair    0.023193
Name: status_group, dtype: float64